In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
df = pd.read_pickle('offensive_comments.df')
df = df.sample(frac=1)

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import RussianStemmer

In [4]:
stop_words = set(stopwords.words('russian'))
stemmer = RussianStemmer()


def preprocess(comment):
    # сначала комментарий преобразуем к нижнему регистру
    # заглавные буквы нам ни к чему
    comment = comment.lower()

    # токенизируем текст, получив список токенов
    tokens = word_tokenize(comment)

    # в списке оставим только те токены, что являются словами (выкинув, например, знаки препинания)
    # также выкинем стоп-слова
    # всё, что осталось, подвергнем стеммизации
    tokens = [stemmer.stem(t) for t in tokens if t.isalpha() and t not in stop_words]

    return tokens

In [5]:
df['comment'] = df['comment'].apply(preprocess)

In [6]:
with open('vectors.w2v', 'rb') as file:
    wv = pickle.load(file)

In [7]:
df['comment'] = df['comment'].apply(lambda x: [wv.vocab[word].index for word in x if word in wv.vocab])

In [8]:
df = df.loc[df['comment'].map(len) > 0]

In [9]:
comments_lengths = df['comment'].map(len)

In [10]:
max_length = int(comments_lengths.quantile(0.95))

In [11]:
padding_code = len(wv.vocab)  # код для нулевого вектора-заполнителя

In [12]:
x = tf.keras.preprocessing.sequence.pad_sequences(df['comment'],
                                                  maxlen=max_length,  # максимальная длина последовательности
                                                  value=padding_code,  # код для заполнения отсутствующих ячеек
                                                  padding='post',  # добавлять заполнитель в конце последовательности
                                                  truncating='post')  # если последовательность длинная, отрезать конец

In [13]:
y = np.array(df['is_offensive'])

In [14]:
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


x_train, x_val, x_test = train_val_test_split(x)
y_train, y_val, y_test = train_val_test_split(y)

In [15]:
# размерность эмбеддинга - длина вектора в нашем word2vec
embedding_dim = len(wv.get_vector('привет'))

# количество слов в словаре - количество строк в матрице
n_words = len(wv.vocab)

# инициализируем матрицу нулями
# сделаем в ней на 1 строку больше, чтобы в последней строке
# остался нулевой ембеддинг для вектора-заполнителя
embedding_matrix = np.zeros((n_words + 1, embedding_dim))

# далее просто каждой строке матрицы присваиваем вектор для слова с соответствующим номером
for word in wv.vocab:
    embedding_matrix[wv.vocab[word].index] = wv.get_vector(word)

In [16]:
embedding_matrix[:, 0].sum()

-5004.540545012491

In [17]:
model = tf.keras.models.Sequential([
    # создаём эмбеддинг-слой соответствущей размерности
    tf.keras.layers.Embedding(n_words + 1,
                              embedding_dim,
                              weights=[embedding_matrix],  # сразу инициализируем веса построенной матрицей
                              trainable=False),  # больше тренировать этот слой не нужно
    
    # внутри "обёртки" Bidirectional уставляем LSTM с нужными настройками
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    
    tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 300)         59927700  
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
dense (Dense)                (None, None, 10)          1290      
Total params: 60,313,502
Trainable params: 385,802
Non-trainable params: 59,927,700
_________________________________________________________________
